In [8]:
import sqlite3
import pandas as pd

# Reabrir a conexão com o banco
conn = sqlite3.connect("../data/melhores_compras.db")

# Agora você pode rodar as consultas
categorias = pd.read_sql_query("SELECT * FROM categoria;", conn)
display(categorias)

produtos = pd.read_sql_query("SELECT id_produto, nome, id_categoria FROM produto;", conn)
display(produtos)

chamados = pd.read_sql_query("SELECT id_produto FROM sgv_ocorrencia_sac;", conn)
display(chamados)

conn.close()


,id_categoria,nome_categoria


,id_produto,nome,id_categoria
0,1,Smartphone XYZ,1
1,2,Smartphone XYZ,2
2,3,Notebook Alpha Pro,3
3,4,Fone de Ouvido Wireless,1
4,5,Smartwatch FitBand,2
5,6,Câmera de Ação GoCam 4K,3
6,7,Console Gamer PlayZ,1
7,8,Caixa de Som Bluetooth BoomSound,2
8,9,Monitor UltraWide 29”,3
9,10,Teclado Mecânico RGB Pro,1


,id_produto
0,1
1,1
2,1


In [9]:
conn = sqlite3.connect("../data/melhores_compras.db")
cursor = conn.cursor()

# Inserir 3 categorias básicas
categorias = [
    (1, 'Smartphones e Eletrônicos'),
    (2, 'Wearables e Acessórios'),
    (3, 'Informática e Games')
]

cursor.executemany("INSERT INTO categoria (id_categoria, nome_categoria) VALUES (?, ?);", categorias)

conn.commit()
conn.close()

print("Categorias inseridas com sucesso.")


Categorias inseridas com sucesso.


In [10]:
# === Consulta DQL: Quantidade de chamados por categoria ===

import sqlite3
import pandas as pd

# 1. Conectar ao banco
conn = sqlite3.connect("../data/melhores_compras.db")
cursor = conn.cursor()

# 2. Verificar se a coluna id_categoria já existe
produto_cols = pd.read_sql_query("PRAGMA table_info(produto);", conn)
if 'id_categoria' not in produto_cols['name'].values:
    cursor.execute("ALTER TABLE produto ADD COLUMN id_categoria INTEGER;")
    conn.commit()

# 3. Preencher id_categoria com dados de exemplo
produto_df = pd.read_sql_query("SELECT * FROM produto", conn)
categorias = [1, 2, 3]
produto_df['id_categoria'] = [categorias[i % len(categorias)] for i in range(len(produto_df))]

# 4. Atualizar a tabela produto no banco
produto_df.to_sql("produto", conn, if_exists="replace", index=False)

# 5. Executar consulta de quantidade de chamados por categoria
query = '''
SELECT 
    c.id_categoria,
    c.nome_categoria,
    COUNT(s.id_produto) AS qtd_chamados
FROM categoria c
LEFT JOIN produto p ON c.id_categoria = p.id_categoria
LEFT JOIN sgv_ocorrencia_sac s ON p.id_produto = s.id_produto
GROUP BY c.id_categoria, c.nome_categoria
ORDER BY qtd_chamados DESC;
'''

resultado = pd.read_sql_query(query, conn)
conn.close()

# 6. Exibir resultado
print("Consulta executada com sucesso. Resultado:")
display(resultado)

# 7. Salvar como evidência
resultado.to_csv("../relatorios/evidencia_chamados_por_categoria.csv", index=False)


Consulta executada com sucesso. Resultado:


,id_categoria,nome_categoria,qtd_chamados
0,1,Smartphones e Eletrônicos,3
1,2,Wearables e Acessórios,0
2,3,Informática e Games,0
